- Lấy ra middlename.
- N-gram cho các middlename này.

In [1]:
import time
import re
import nltk
import numpy as np
import pandas as pd

In [2]:
# Loading dataset
file = pd.read_csv("VNName_Final.csv", encoding='utf-8')

In [3]:
# Số lượng dòng, cột của file csv
print(file.shape)

(1147651, 2)


In [4]:
name = file['full_name']
label = file['gender']

In [5]:
# Tiền xử lý dữ liệu cho full name
male_names = ''
female_names = ''
words = 0

for sen in range(0,len(name)):
    
    # Dòng này đếm số lượng ký tự
    #words += len(str(name[sen]).split())
    
    if (label[sen] == 1) and (len(str(name[sen]).split()) > 2):
        
        female_name = re.sub(r'\W', ' ', str(name[sen])) 
        # Dòng này dùng để loại bỏ Họ, Tên ra khỏi document (chỉ lấy middlename)
        female_name = female_name.split(' ',1)[1]
        female_name = female_name.rsplit(' ', 1)[0]
        female_name = female_name.strip()    
    
        # Converting to Lowercase
        female_name = female_name.lower()
        
        if female_name is not '': 
            female_names = female_names + ' ' + female_name

    if (label[sen] == 0) and (len(str(name[sen]).split()) > 2):
    
        # Remove all the special characters
        male_name = re.sub(r'\W', ' ', str(name[sen]))
        # Dòng này dùng để loại bỏ Họ, Tên ra khỏi document
        male_name = male_name.split(' ',1)[1]
        male_name = male_name.rsplit(' ', 1)[0]
        male_name = male_name.strip()   
       
    
        # Converting to Lowercase
        male_name = male_name.lower()
        
        if male_name is not '':    
            male_names = male_names + ' ' + male_name
        
        

In [6]:
male_names = male_names.split()
female_names = female_names.split()

#print(male_names)
#print(female_names)

In [7]:
len(male_names)
len(female_names)

973615

In [8]:
#Tạo featureset là tất cả các từ trong middlename của male và female
featuresets = []

for i in range(0,len(female_names)):
    a = (female_names[i],'female')
    featuresets.append(a)
    
for i in range(0,len(male_names)):
    a = (male_names[i],'male')
    featuresets.append(a)
    
#print(featuresets)

In [9]:
import random 
random.shuffle(featuresets) 

In [10]:
#print(featuresets)

In [11]:
X = []
Y = []
for i in range(0,len(featuresets)):
    X.append(featuresets[i][0])
    Y.append(featuresets[i][1])
    
#print(X)
#print(Y)

In [12]:
len(Y)

1766961

In [14]:
# Using 1-grams and 2-grams 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

bigram_vectorizer = CountVectorizer(analyzer="word",ngram_range=(1, 1),token_pattern=r'\b\w+\b', min_df=1,max_features=300)

X = bigram_vectorizer.fit_transform(X).toarray()

# Splitting the dataset into the Training set and Test set
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.4, random_state = 0)

In [17]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=1)
classifier_LR.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn Logistic Regression: ", str(end-start), " giây")

C:\Users\admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Thời gian huấn Logistic Regression:  44.76429009437561  giây


In [18]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes
from sklearn.naive_bayes import BernoulliNB

start = time.time()

classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện Naive Bayes: ", str(end-start), " giây")

Thời gian huấn luyện Naive Bayes:  12.781916379928589  giây


In [19]:
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start = time.time()

classifier_RF = RandomForestClassifier(n_estimators=100, random_state=0)
classifier_RF.fit(X_Train, Y_Train) 

end = time.time()


print("Thời gian huấn luyện Random Forest: ", str(end-start), " giây")

Thời gian huấn luyện Random Forest:  4202.073364019394  giây


In [119]:
# Xây dựng mô hình huấn luyện bằng KNN

from sklearn.neighbors import KNeighborsClassifier

start = time.time()

classifier_KNN = KNeighborsClassifier(n_neighbors=5)
classifier_KNN.fit(X_Train, Y_Train)

end = time.time()

print("Thời gian huấn luyện KNN: ", str(end-start), " giây")

Thời gian huấn luyện KNN:  0.0019941329956054688  giây


In [20]:
# Predicting the test set results with LogisticRegresion
Y_Pred_LR = classifier_LR.predict(X_Test)

# Predicting the test set results with Naive Bayes
Y_Pred_NB = classifier_NB.predict(X_Test)

# Predicting the test set results with RF 
Y_Pred_RF = classifier_RF.predict(X_Test)

#Y_Pred_KNN = classifier_KNN.predict(X_Test)

In [21]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Accuracy LogisticRegression: ', accuracy_score(Y_Test, Y_Pred_LR))
print('Accuracy Naive Bayes: ', accuracy_score(Y_Test,Y_Pred_NB))
print('Accuracy RF: ', accuracy_score(Y_Test, Y_Pred_RF))
#print('Accuracy KNN: ', accuracy_score(Y_Test, Y_Pred_KNN))

Accuracy LogisticRegression:  0.7595520561415423
Accuracy Naive Bayes:  0.7543482105590809
Accuracy RF:  0.7595520561415423
